<a href="https://colab.research.google.com/github/ffg-kom/mapping_evolution/blob/master/getting_data_ready.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

The following code was copied from https://colab.research.google.com/github/nestauk/im_tutorials/blob/master/notebooks/01_intro_to_pandas_tutorial.ipynb.

In [1]:
%load_ext autoreload
%autoreload 2
# install im_tutorial package
!pip install git+https://github.com/nestauk/im_tutorials.git

  Cloning https://github.com/nestauk/im_tutorials.git to /tmp/pip-req-build-zc0h1xai
  Running command git clone -q https://github.com/nestauk/im_tutorials.git /tmp/pip-req-build-zc0h1xai
  Created wheel for im-tutorials: filename=im_tutorials-0.1.1-cp36-none-any.whl size=15103 sha256=92e91cad0eb6ee4addfa2387327c4dda423190d004e050088169dd06e70e0ce5
  Stored in directory: /tmp/pip-ephem-wheel-cache-lp313cnr/wheels/47/a3/cb/bdc5f9ba49bcfd2c6864b166a1566eb2f104113bf0c3500330
Successfully built im-tutorials


In [0]:
# numpy for mathematical functions
import numpy as np
# pandas for handling tabular data
import pandas as pd
# explained later
from im_tutorials.data.cordis import h2020_projects

# for plotting
import matplotlib.pyplot as plt

# data
from im_tutorials.data.cordis import cordis_table

# Loading data

In [0]:
# load tables
cordis_o_df = cordis_table('organisations')
cordis_p_df = cordis_table('projects')
cordis_p_o_df = cordis_table('project_organisations')

# merge them
cordis_project_orgs_df = cordis_p_o_df.merge(
  cordis_o_df, left_on='organization_id', right_on='id', how='left'
)
cordis_project_orgs_df = cordis_project_orgs_df.merge(
  cordis_p_df, left_on='project_rcn', right_on='rcn', how='left'
)


In [55]:
# add number of partners per project as a column
count_partners_df = cordis_project_orgs_df[['project_rcn', 'organization_id']].groupby( by='project_rcn' ).count()
#cordis_project_orgs_df = cordis_p_o_df.merge(
#  count_partners_df, on='project_rcn', how='left'
#)


# display header
#cordis_project_orgs_df.head()
count_partners_df.head()

,organization_id
project_rcn,
85231,5
85232,7
85233,6
85234,11
85235,9


# Playing with the data

In [25]:
cordis_project_orgs_df.columns

Index(['project_rcn', 'organization_id', 'activity_type', 'address',
       'contribution', 'type', 'website_x', 'id', 'name', 'country_code',
       'country_name', 'rcn', 'acronym', 'end_date_code', 'ec_contribution',
       'framework', 'funding_scheme', 'funded_under', 'objective',
       'project_description', 'start_date_code', 'status', 'title',
       'total_cost', 'website_y'],
      dtype='object')

In [26]:
cordis_project_orgs_df['activity_type'].value_counts()

Higher or Secondary Education Establishments                                                         88335
Private for-profit entities (excluding Higher or Secondary Education Establishments)                 78810
Research Organisations                                                                               55099
Other                                                                                                15406
Public bodies (excluding Research Organisations and Secondary or Higher Education Establishments)    12454
                                                                                                       936
Name: activity_type, dtype: int64

In [42]:
subset = cordis_project_orgs_df[
    (cordis_project_orgs_df.activity_type == 'Private for-profit entities (excluding Higher or Secondary Education Establishments)')
  & (cordis_project_orgs_df.framework == 'H2020')
]

# within subset, how often do organisations appear
subset['organization_id'].value_counts()



999993856    148
999960488     84
999909854     79
999951467     76
999908787     73
999987260     72
999812854     62
999952243     60
999959130     58
999751938     58
999792484     57
999910824     57
998398206     54
999926829     52
999991819     52
999908205     50
998627417     47
953176030     44
999977657     44
999991625     43
999787343     42
999702371     41
999908981     41
999971934     40
916274126     40
999963010     38
999754557     38
999941379     38
999908496     37
999978918     36
            ... 
937962550      1
920038502      1
918957922      1
939118208      1
934671437      1
950781003      1
921019463      1
906933317      1
915782724      1
929399875      1
928027713      1
968662080      1
997485630      1
952517691      1
916581422      1
995947501      1
938144813      1
927923244      1
997381161      1
905055494      1
926985254      1
918318110      1
938040344      1
920237061      1
951732476      1
904743930      1
999704505      1
916988919     

In [43]:
subset[['organization_id', 'start_date_code', 'funding_scheme']]

,organization_id,start_date_code,funding_scheme
135899,999905974,2014-01-01,CSA - Coordination and support action
135943,999574428,2014-08-01,CSA - Coordination and support action
135953,999940118,2014-08-01,CSA - Coordination and support action
135972,984286452,2014-10-01,CSA - Coordination and support action
135977,998142126,2014-10-01,CSA - Coordination and support action
135982,998860896,2014-10-01,CSA - Coordination and support action
136004,999641940,2014-10-01,CSA - Coordination and support action
136009,999773666,2014-10-01,CSA - Coordination and support action
136018,999939730,2014-10-01,CSA - Coordination and support action
136029,937991165,2015-02-01,RIA - Research and Innovation action
